In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
from concurrent.futures import ThreadPoolExecutor, as_completed
import dask.dataframe as dd
import sys
import glob
import os

sys.path.append("/global/scratch/projects/fc_mvslab/OpenProjects/Sanjana/TREBL/")

from scripts import initial_map, map_refiner, complexity, finder, preprocess, error_correct, plotting, umi_deduplicate


import glob
from tqdm import tqdm  # progress bar

How many colonies (unique AD-AD_BC-RPTR_BC combos) are in the Gcn4 UMI experiment?

How many reads did we get for each of the timepoint/replicates?

Max says that the 180 and 240 timepoints are undersequenced - are all the other timepoints good? (what is the acceptable #reads for each timepoint?)
this is all mostly for the RPTR reads, but it would be helpful to know the average reads needed for an AD timepoint/replicate also